# Median Metrics Tables

F1/FX

This notebook was written by Logan Qualls. Data for this work is sourced from the National Center for Atmospheric Research's Catchment Attributes and Meterology for Large-Sample Studies (CAMELS) dataset, and this notebook is designed to work specifically with Frederik Kratzert's NeuralHydrology (NH; https://github.com/neuralhydrology/neuralhydrology) and Grey Nearing's SACSMA-SNOW17 (SAC-SMA; https://github.com/Upstream-Tech/SACSMA-SNOW17). NH provides a flexible framework with a variety of tools specifically designed for straightforward application of Long Short-Term Memory networks to hydrological modeling. The SACSMA-SNOW17 model provides a Python interface for the SAC-SMA model.

This notebook compiles all the median metrics for all avaliable models into several tables by type (extreme or random) and ultimately combines them for export.

### Import Libraries

In [1]:
import os
import numpy as np
import pandas as pd
import pickle as pkl
from pathlib import Path
import matplotlib.pyplot as plt

Here I remove the display limit for the rows and columns for Pandas DataFrames.

In [2]:
#View all columns
pd.set_option('display.max_columns', None)
#View all rows
pd.set_option('display.max_rows', None)

### Define Parameters

The only parameters necessary for this notebook are the desired climate index and the path to the notebook_env_saves folder, if that folder has been moved from the original folder structure.



##### Notebook Configurations

In [4]:
#########################################################################################

#Define index we want to create tables and histograms for (currently only supports one)
index = 'aridity'

#########################################################################################

##### Paths

In [3]:
#########################################################################################

#Path to working directory (current directory)
working_dir = Path(os.getcwd())

#Path to notebook_env_saves directory (.../notebook_env_saves)
env_saves_dir = working_dir / 'notebook_env_saves'

#########################################################################################

### Load Source Data

First we load in our source data. This cell automatically retrieves a list of files starting with 'basin', indicating that the file contains per-basin metrics (as opposed to overall, as for the case in the files starting with 'cdfs') for a model. Assuming you have ensembled and analyzed a model prior to running this notebook, it will be included in the resulting tables.

Here we make a dictionary containing the per-basin metrics for each model.

In [19]:
#Retreive list of experiments with existing basin_metrics_files
metrics_list = [x for x in os.listdir(env_saves_dir) if x.startswith('basin')]

#Initiate dictionary to store all loaded cdfs files
metrics_dict = {}

#For every file in cdfs_list...
for file in metrics_list:
    
    #Define path to the file
    path = env_saves_dir / file
    
    #Get name from file name
    name = file.split('basin_metrics_')[1].split('.')[0]
    
    #Open path...
    with open(path,'rb') as f:
        
        #...and load
        metrics_dict[name] = pkl.load(f)

Here we make lists detailing what models are extreme and what models are random, as well as what types of experiments exist for each. Here we exclude the NWM benchmark runs -- they will be added later.

In [6]:
#Retrieve list of loaded models
models = list(metrics_dict.keys())
#If a model does not contain 'random' in the name, it is an extreme experiment
ext_models = [model for model in models if not 'random' in model and 'nwm_' not in model]
#If a model contains 'random' in the name, it is a random experiment
rand_models = [model for model in models if 'random' in model and 'nwm_' not in model]

In [7]:
#Retrieve the keys for an example extreme model, which constitutes the avaliable extreme experiments
ext_exps = list(metrics_dict[ext_models[0]].keys())
#Only use experiments for the defined climate index
ext_exps = [experiment for experiment in ext_exps if index in experiment]
#Retrieve the keys for an example random model, which constitutes the avaliable random experiments
rand_exps = list(metrics_dict[rand_models[0]].keys())
#Only use experiments for the defined climate index
metrics = list(metrics_dict[models[0]][ext_exps[0]].columns)

### Median Metrics Tables

##### Extreme Model Table

In [20]:
#Create multiindex for extreme models and experiments
exp_multidex = pd.MultiIndex.from_product([ext_models,ext_exps])

#Create dataframe with extreme multiindex
ext_median_df = pd.DataFrame(columns=metrics,index=exp_multidex)

#For every extreme model...
for model in ext_models:
    
    #Get a list of that model's experiments
    experiments = list(metrics_dict[model].keys())
    
    #And for each of those experiments...
    for experiment in experiments:
        
        #If the defined index is in the experiment...
        if index in experiment:
            
            #Get a list of metrics
            metrics = list(metrics_dict[model][experiment].columns)
            
            #And for each of those metrics...
            for metric in metrics:
                
                #Calculate the median value of that metric and save it into ext_median_df
                ext_median_df.loc[model,experiment][metric] = np.median(metrics_dict[model][experiment][metric])

#Show extreme model median metrics table
ext_median_df

alpha_nse  beta_kge  \
nh_dynamic_extreme_daymet_nwm         aridity_high  0.868716  0.987979   
                                      aridity_low   0.870547  0.973161   
nh_static_extreme_daymet_nwm          aridity_high  0.878733  1.022305   
                                      aridity_low   0.856995  0.937718   
nh_static_extreme_daymet_all          aridity_high  0.856041  1.044905   
                                      aridity_low   0.830512  0.944697   
sacsma_extreme_daymet_all             aridity_high  0.797808  1.292733   
                                      aridity_low   0.764989  1.121742   
sacsma_extreme_daymet_nwm             aridity_high  0.818521  1.316392   
                                      aridity_low   0.791821  1.143222   
nh_dynamic_extreme_daymet_all         aridity_high  0.854783  1.012186   
                                      aridity_low   0.851551  0.991949   
nh_static_extreme_nldas_extended_nwm  aridity_high  0.839508  0.995585   
                                      aridity_low   0.852301  0.976854   
nh_dynamic_extreme_nldas_extended_nwm aridity_high  0.834456  0.962952   
                                      aridity_low   0.858816  0.993235   
nh_static_extreme_nldas_extended_all  aridity_high  0.812359  0.997395   
                                      aridity_low   0.848864  0.978076   
nh_dynamic_extreme_nldas_extended_all aridity_high  0.812208  0.953166   
                                      aridity_low   0.840401  1.024379   
sacsma_extreme_nldas_extended_nwm     aridity_high  0.838278  1.339763   
                                      aridity_low   0.776111  1.126517   
sacsma_extreme_nldas_extended_all     aridity_high  0.842018  1.318418   
                                      aridity_low   0.756517  1.091747   

                                                    beta_nse       kge  \
nh_dynamic_extreme_daymet_nwm         aridity_high -0.004954  0.727424   
                                      aridity_low  -0.012992  0.758301   
nh_static_extreme_daymet_nwm          aridity_high  0.015431  0.746649   
                                      aridity_low   -0.03082   0.75456   
nh_static_extreme_daymet_all          aridity_high  0.026248  0.680178   
                                      aridity_low  -0.032531  0.728444   
sacsma_extreme_daymet_all             aridity_high  0.163788  0.398831   
                                      aridity_low   0.074366  0.554993   
sacsma_extreme_daymet_nwm             aridity_high  0.166807  0.435189   
                                      aridity_low   0.084958  0.558894   
nh_dynamic_extreme_daymet_all         aridity_high  0.007774  0.676715   
                                      aridity_low   -0.00366  0.734125   
nh_static_extreme_nldas_extended_nwm  aridity_high  0.004634  0.690095   
                                      aridity_low  -0.008603  0.750757   
nh_dynamic_extreme_nldas_extended_nwm aridity_high  -0.01509  0.680836   
                                      aridity_low  -0.001951  0.739194   
nh_static_extreme_nldas_extended_all  aridity_high  0.003128  0.666491   
                                      aridity_low   -0.01274  0.729693   
nh_dynamic_extreme_nldas_extended_all aridity_high -0.026251  0.639179   
                                      aridity_low   0.014339  0.723598   
sacsma_extreme_nldas_extended_nwm     aridity_high  0.202517  0.379773   
                                      aridity_low   0.073962  0.554618   
sacsma_extreme_nldas_extended_all     aridity_high  0.203882  0.389674   
                                      aridity_low   0.055769  0.567145   

                                                         mse       nse  \
nh_dynamic_extreme_daymet_nwm         aridity_high  0.494415  0.736661   
                                      aridity_low   2.088403  0.748234   
nh_static_extreme_daymet_nwm          aridity_high  0.443082  0.741288   
                                      aridity_low   2

##### Random Model Table (Median)

Here we create a table detailing the each **median** metric score for the 531 CAMELS basins. 

In [17]:
#Create multiindex for random models and experiments
rand_multidex = pd.MultiIndex.from_product([rand_models,rand_exps],names=['models','experiments'])

#Create dataframe with random multiindex
rand_median_df = pd.DataFrame(columns=metrics,index=rand_multidex)

#For every random model...
for model in rand_models:
    
    #Get a list of experiments
    experiments = list(metrics_dict[model].keys())
    
    #And for each of those experiments...
    for experiment in experiments:
        
        #And for each metric...
        for metric in metrics:
            
            #Calculate the median value of that metric and save it into ext_median_df
            rand_median_df.loc[model,experiment][metric] = np.median(metrics_dict[model][experiment][metric])

#Peek at rand_median_df
rand_median_df

alpha_nse  beta_kge  \
models                               experiments                       
nh_dynamic_random_daymet_all         0            0.890283  0.987786   
                                     1             0.88598  0.987089   
                                     2            0.904534  0.995917   
                                     3            0.895389  0.980272   
                                     4            0.891942  0.990021   
nh_static_random_nldas_extended_nwm  0            0.913115  0.989114   
                                     1            0.910209  0.991753   
                                     2            0.919222   0.99605   
                                     3            0.921877  0.995308   
                                     4            0.921079  0.998723   
nh_dynamic_random_nldas_extended_nwm 0            0.900364  0.990108   
                                     1            0.902555  0.991081   
                                     2            0.910142  0.997447   
                                     3            0.914457  0.992481   
                                     4            0.916011  0.995575   
sacsma_random_daymet_all             0            0.821996  1.154499   
                                     1            0.804923  1.146915   
                                     2            0.794874  1.142568   
                                     3             0.81638  1.134395   
                                     4            0.807987  1.152825   
nh_static_random_daymet_all          0            0.900818  0.992008   
                                     1            0.878138  0.982538   
                                     2            0.878976  0.981917   
                                     3            0.908227    0.9944   
                                     4            0.891627  0.989448   
nh_static_random_daymet_nwm          0             0.91311  0.983051   
                                     1            0.917152  0.993822   
                                     2            0.912448  0.990318   
                                     3            0.921474  0.990153   
                                     4            0.921452  0.992796   
nh_static_random_nldas_extended_all  0              0.8959  0.982664   
                                     1             0.88347  0.984332   
                                     2            0.894287  0.974267   
                                     3             0.87621  0.988437   
                                     4             0.89156  0.992502   
nh_dynamic_random_daymet_nwm         0               0.924  0.998836   
                                     1             0.91509   0.99621   
                                     2            0.923174  0.985991   
                                     3            0.923108  0.995312   
                                     4            0.918566  0.996235   
nh_dynamic_random_nldas_extended_all 0            0.883626  0.992303   
                                     1            0.898575  1.001952   
                                     2            0.885901  0.990877   
                                     3            0.896953  0.990613   
                                     4            0.891244  0.993038   
sacsma_random_daymet_nwm             0            0.807407  1.164616   
                                     1            0.803828  1.154958   
                                     2            0.805162   1.18384   
                                     3            0.814591  1.178965   
                                     4            0.815133  1.169935   
sacsma_random_nldas_extended_nwm     0            0.807407  1.164616   
                                     1            0.803828  1.154958   
                                     2            0.805162   1.18384   
                                     3            0.814591  1.178965   
                 

##### Random Model Table (Mean of the Medians)

Here we take the mean of the median metric scores by model so that we have one set of descriptive metrics for each random model.

In [21]:
mean_rand_df = pd.DataFrame(columns = rand_median_df.columns,index=rand_models)

for model in rand_models:
    rand_list = []
    mean_rand_df.loc[model] = rand_median_df.loc[model].mean()

mean_rand_df

,alpha_nse,beta_kge,beta_nse,kge,mse,nse,rmse,pearsonr
nh_dynamic_random_daymet_all,0.893626,0.988217,-0.005372,0.781075,1.128091,0.773443,1.061908,0.893876
nh_static_random_nldas_extended_nwm,0.9171,0.994189,-0.002159,0.865654,0.713633,0.862731,0.844622,0.932401
nh_dynamic_random_nldas_extended_nwm,0.908706,0.993338,-0.002495,0.838795,0.841304,0.834425,0.916966,0.919115
sacsma_random_daymet_all,0.809232,1.14624,0.081949,0.579378,2.003844,0.59443,1.415194,0.812554
nh_static_random_daymet_all,0.891557,0.988062,-0.005727,0.79685,1.062771,0.78263,1.030707,0.897106
nh_static_random_daymet_nwm,0.917127,0.990028,-0.004023,0.855282,0.797778,0.845708,0.893057,0.924292
nh_static_random_nldas_extended_all,0.888286,0.98444,-0.006865,0.779183,1.158393,0.767198,1.076099,0.889655
nh_dynamic_random_daymet_nwm,0.920788,0.994517,-0.001912,0.853988,0.773893,0.84424,0.879584,0.924546
nh_dynamic_random_nldas_extended_all,0.89126,0.993757,-0.0017,0.769088,1.186219,0.757388,1.088953,0.884685
sacsma_random_daymet_nwm,0.809224,1.170463,0.092949,0.59549,1.773378,0.660777,1.331648,0.841864


##### Total Table

Here we concatenate the extreme and (mean) random median metrics tables into one big table.

In [15]:
#Combine the extreme and random median dfs
median_df = pd.concat([ext_median_df,mean_rand_df], ignore_index=False)

#Peek at combined df
median_df

,alpha_nse,beta_kge,beta_nse,kge,mse,nse,rmse,pearsonr
"(nh_dynamic_extreme_daymet_nwm, aridity_high)",0.868716,0.987979,-0.004954,0.727424,0.494415,0.736661,0.703147,0.876347
"(nh_dynamic_extreme_daymet_nwm, aridity_low)",0.870547,0.973161,-0.012992,0.758301,2.088403,0.748234,1.445131,0.876207
"(nh_static_extreme_daymet_nwm, aridity_high)",0.878733,1.022305,0.015431,0.746649,0.443082,0.741288,0.665644,0.879823
"(nh_static_extreme_daymet_nwm, aridity_low)",0.856995,0.937718,-0.03082,0.75456,2.082557,0.750663,1.443107,0.880899
"(nh_static_extreme_daymet_all, aridity_high)",0.856041,1.044905,0.026248,0.680178,0.432945,0.710859,0.657986,0.87075
"(nh_static_extreme_daymet_all, aridity_low)",0.830512,0.944697,-0.032531,0.728444,2.867281,0.71887,1.693305,0.870274
"(sacsma_extreme_daymet_all, aridity_high)",0.797808,1.292733,0.163788,0.398831,0.822808,0.485594,0.907088,0.781154
"(sacsma_extreme_daymet_all, aridity_low)",0.764989,1.121742,0.074366,0.554993,5.10249,0.534149,2.258869,0.795654
"(sacsma_extreme_daymet_nwm, aridity_high)",0.818521,1.316392,0.166807,0.435189,0.97274,0.497586,0.986276,0.784501
"(sacsma_extreme_daymet_nwm, aridity_low)",0.791821,1.143222,0.084958,0.558894,3.976123,0.558634,1.994022,0.80219
